In [ ]:

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [ ]:

# Carregar as tabelas exportadas do PostgreSQL
tb_gastos = pd.read_csv('tb_gastos.csv')
tb_deputados = pd.read_csv('tb_deputados.csv')

# Ajustar os nomes das colunas para clareza
tb_gastos.columns = ['id_gasto', 'id_deputado', 'data', 'tipo_despesa', 'valor']
tb_deputados.columns = ['estado', 'id_deputado', 'nome', 'partido']


In [ ]:

# Unir as tabelas com base no id do deputado
df = pd.merge(tb_gastos, tb_deputados, on='id_deputado', how='left')

# Selecionar as colunas relevantes para a análise
df_ml = df[['nome', 'partido', 'estado', 'tipo_despesa', 'valor']].copy()

# Codificar variáveis categóricas para aplicação de algoritmos de ML
le = LabelEncoder()
for col in ['nome', 'partido', 'estado', 'tipo_despesa']:
    df_ml[col] = le.fit_transform(df_ml[col])


In [ ]:

# Separar features e target para Árvore de Decisão
X = df_ml[['partido', 'estado', 'tipo_despesa']]
y = df_ml['valor']

# Treinar Árvore de Decisão
tree_model = DecisionTreeRegressor(max_depth=4, random_state=42)
tree_model.fit(X, y)

# Plotar a Árvore de Decisão
plt.figure(figsize=(12, 8))
plot_tree(tree_model, feature_names=['partido', 'estado', 'tipo_despesa'], filled=True, rounded=True)
plt.title("Árvore de Decisão - Fatores que Influenciam o Valor dos Gastos")
plt.show()


In [ ]:

# Aplicar KMeans para clusterização de padrões de gastos
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_ml['cluster'] = kmeans.fit_predict(X)

# Plotar os clusters
plt.figure(figsize=(8, 6))
scatter = plt.scatter(df_ml['tipo_despesa'], df_ml['valor'], c=df_ml['cluster'], cmap='viridis')
plt.xlabel('Tipo de Despesa (codificado)')
plt.ylabel('Valor do Gasto')
plt.title('Clusterização dos Gastos Parlamentares (KMeans)')
plt.colorbar(scatter, label='Cluster')
plt.show()
